# Working with GeoDataFrames

This exercise is adapted from Kelsey Jordahl's SciPy tutorial, available here: 

https://github.com/kjordahl/SciPy-Tutorial-2015


In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gp
from shapely.geometry import Point
import pandas as pd

NYC borough boundaries downloaded from [Bytes of the Big Apple](http://www.nyc.gov/html/dcp/download/bytes/nybb_13a.zip)

We can easily read a shapefile. 

In [ ]:
# we can easily read a shapefile
boros = gp.read_file('nybb_15b/nybb.shp')
boros

We see that there are a few attributes, and that the geometry field contains the boundaries of each polygon. 

In [ ]:
# set the index to be the name of the borough, and sort in order of the borough code
boros.reset_index(inplace=True)
boros.set_index('BoroName', inplace=True)
boros.sort_values(by='BoroCode', inplace=True)
boros

In [ ]:
# we can easily plot these. 
p = boros.plot()

In [ ]:
# or change the size of the figure
boros.plot(figsize=(10, 10))

Lots of style options are available.  Anything that can by passed to pyplot works: 

http://matplotlib.org/api/pyplot_summary.html

In [ ]:
# if we want to overlay two maps, we create the first, and pass it 
# as the ax option to the second. 

base = boros.plot(figsize=(10, 10),alpha=0.0)

# here we are doing some geometric operations
boros.geometry.convex_hull.plot(ax=base, figsize=(10, 10))

In [ ]:
# we can calculate buffers
# the negative number indicates an inside buffer
# Here let's create a de-militirized zone between the boroughs

eroded = boros.geometry.buffer(-5280)
base = boros.plot(alpha=0.0, figsize=(10, 10))
eroded.plot(ax=base, figsize=(10, 10))

In [ ]:
# the area attribute is automatic, so we can easily see the difference
eroded.area

In [ ]:
boros.geometry.area

In [ ]:
# calculate the fraction of the borough that is inland from the others

inland = eroded.area / boros.geometry.area
boros['inland_fraction'] = inland
boros

Let's create a normal pandas `Series` with population values for each borough from the 2010 census.

In [ ]:
population = pd.Series({'Manhattan': 1585873, 'Bronx': 1385108, 'Brooklyn': 2504700,
                     'Queens': 2230722, 'Staten Island': 468730})
population

In [ ]:
boros['population'] = population
boros

In [ ]:
# calculate the population density in people per square mile
boros['pop_density'] = boros['population'] / boros.geometry.area * 5280 ** 2
boros.sort(columns='pop_density', ascending=False)

In [ ]:
# if we specify a column name, we can make a chloropleth
#Chloropleth maps are maps where the color of each shape is based on the value of an associated variable. 

boros.plot(column='pop_density', figsize=(10, 10))

In [ ]:
# we can save it to a shapefile, or to a geojson file
# note that a geojson is a simpler data format, so may be easier to work with

boros.to_file('boros.shp')
boros.to_file('boros.json', driver="GeoJSON")

## Interactive maps

We can easily send geopandas data to folium to make interactive maps 

In [ ]:
import folium

# to do this we convert to a geojson object
# this contains all the necessary information
# we'll come back to the need for the to_crs conversion
gjson = boros.to_crs(epsg='4326').to_json()

# create the map
m = folium.Map([40.730610,-73.935242],
                tiles='Mapbox Bright',
                zoom_start=10)

# add the layer
folium.GeoJson(gjson,
               name='Boroughs'
              ).add_to(m)

# This lets us turn the layer on or off
folium.LayerControl().add_to(m)

# save it
m.save('boros.html')

## Working with projections

In the above, you noticed the to_crs() method.  This converts the data to a different coordinate reference system (CRS). This is a seris of parameters that defines the coordinate system and spatial extent of some data set. 

Basically, it is about going from a round earth to a flat map in various different ways, and for various different parts of the globe.  There are a bunch of these, but only a few seem to be commonly used.  

If you get points that don't show up on a map, or show up in the wrong place, it is often because your two map layers have a different CRS and you need to convert.  The codes and defintitions are all here: 

http://spatialreference.org/